In [2]:
# import libraries
# %matplotlib notebook
# %matplotlib inline

print('Library versions:')

import keras
print(f'keras:{keras.__version__}')
import pandas as pd
print(f'pandas:{pd.__version__}')
import sklearn
print(f'sklearn:{sklearn.__version__}')
import nltk
print(f'nltk:{nltk.__version__}')
import numpy as np
print(f'numpy:{np.__version__}')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
import demoji
import pickle
import re
import random
import time
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize
import tensorflow as tf
print(f'tensofrlow:{tf.__version__}')

import os
os.getcwd()

demoji.__version__
with open(r'../Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
    Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}
    
from tqdm.notebook import tqdm as tqdm
from tqdm.notebook import tqdm
tqdm.pandas()

import pickle

Library versions:
keras:2.8.0
pandas:1.3.4
sklearn:0.24.2
nltk:3.6.7
numpy:1.20.3
tensofrlow:2.8.0


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JiatingChen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
with open('training_data.pickle', 'rb') as handle:
    training_data = pickle.load(handle)
    
with open('testing_data.pickle', 'rb') as handle:
    testing_data = pickle.load(handle)

# model parameters

In [4]:
# 8192 - large enough for demonstration, larger values make network training slower
MAX_VOCAB_SIZE = 2**13
# seq2seq generally relies on fixed length message vectors - longer messages provide more info
# but result in slower training and larger networks
MAX_MESSAGE_LEN = 30  
# Embedding size for words - gives a trade off between expressivity of words and network size
EMBEDDING_SIZE = 100
# Embedding size for whole messages, same trade off as word embeddings
CONTEXT_SIZE = 100
# Larger batch sizes generally reach the average response faster, but small batch sizes are
# required for the model to learn nuanced responses.  Also, GPU memory limits max batch size.
BATCH_SIZE = 4
# Helps regularize network and prevent overfitting.
DROPOUT = 0.2
# High learning rate helps model reach average response faster, but can make it hard to 
# converge on nuanced responses
LEARNING_RATE=0.005

# Tokens needed for seq2seq
UNK = 0  # words that aren't found in the vocab
PAD = 1  # after message has finished, this fills all remaining vector positions
START = 2  # provided to the model at position 0 for every response predicted

# Implementaiton detail for allowing this to be run in Kaggle's notebook hardware
SUB_BATCH_SIZE = 1000

# data preparation

In [25]:
count_vec = CountVectorizer(tokenizer=casual_tokenize, max_features=MAX_VOCAB_SIZE - 3)
print("Fitting CountVectorizer on X and Y text data...")
count_vec.fit(tqdm(training_data.loc[:,'text_x'].to_list() + training_data.loc[:,'text_y'].to_list()))
analyzer = count_vec.build_analyzer()
vocab = {k: v + 3 for k, v in count_vec.vocabulary_.items()}
vocab['__unk__'] = UNK
vocab['__pad__'] = PAD
vocab['__start__'] = START

# Used to turn seq2seq predictions into human readable strings
reverse_vocab = {v: k for k, v in vocab.items()}
print(f"Learned vocab of {len(vocab)} items.")

Fitting CountVectorizer on X and Y text data...


  0%|          | 0/190632 [00:00<?, ?it/s]

Learned vocab of 8192 items.


In [27]:
def to_word_idx(sentence):
    full_length = [vocab.get(tok, UNK) for tok in analyzer(sentence)] + [PAD] * MAX_MESSAGE_LEN
    return full_length[:MAX_MESSAGE_LEN]

def from_word_idx(word_idxs):
    return ' '.join(reverse_vocab[idx] for idx in word_idxs if idx != PAD).strip()

In [38]:
x_text = training_data.loc[:,'text_x']
y_text = training_data.loc[:,'text_y']
print("Calculating word indexes for X...")
train_x = pd.np.vstack(x_text.progress_apply(to_word_idx).values)
print("Calculating word indexes for Y...")
train_y = pd.np.vstack(y_text.progress_apply(to_word_idx).values)

Calculating word indexes for X...


C:\Users\JiatingChen\AppData\Local\Temp\ipykernel_16740\2392148596.py:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  train_x = pd.np.vstack(x_text.progress_apply(to_word_idx).values)


  0%|          | 0/95316 [00:00<?, ?it/s]

Calculating word indexes for Y...


C:\Users\JiatingChen\AppData\Local\Temp\ipykernel_16740\2392148596.py:6: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  train_y = pd.np.vstack(y_text.progress_apply(to_word_idx).values)


  0%|          | 0/95316 [00:00<?, ?it/s]

In [39]:
x_text = testing_data.loc[:,'text_x']
y_text = testing_data.loc[:,'text_y']
print("Calculating word indexes for X...")
test_x = pd.np.vstack(x_text.progress_apply(to_word_idx).values)
print("Calculating word indexes for Y...")
test_y = pd.np.vstack(y_text.progress_apply(to_word_idx).values)

Calculating word indexes for X...


C:\Users\JiatingChen\AppData\Local\Temp\ipykernel_16740\3735564977.py:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  test_x = pd.np.vstack(x_text.progress_apply(to_word_idx).values)


  0%|          | 0/23829 [00:00<?, ?it/s]

Calculating word indexes for Y...


C:\Users\JiatingChen\AppData\Local\Temp\ipykernel_16740\3735564977.py:6: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  test_y = pd.np.vstack(y_text.progress_apply(to_word_idx).values)


  0%|          | 0/23829 [00:00<?, ?it/s]

# Model Structure

In [40]:
# keras imports, because there are like... A million of them.
from keras.models import Model
# from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Input, LSTM, Dropout, Embedding, RepeatVector, concatenate, \
     TimeDistributed
from keras.utils import np_utils

# from tensorflow.keras import Model, Sequential
# from tensorflow.keras.layers import Dense, Input, LSTM, Dropout, Embedding, RepeatVector, concatenate, \
#      TimeDistributed

In [41]:
def create_model():
    shared_embedding = Embedding(
        output_dim=EMBEDDING_SIZE,
        input_dim=MAX_VOCAB_SIZE,
        input_length=MAX_MESSAGE_LEN,
        name='embedding',
    )
    
    # ENCODER
    
    encoder_input = Input(
        shape=(MAX_MESSAGE_LEN,),
        dtype='int32',
        name='encoder_input',
    )
    
    embedded_input = shared_embedding(encoder_input)
    
    # No return_sequences - since the encoder here only produces a single value for the
    # input sequence provided.
    encoder_rnn = LSTM(
        CONTEXT_SIZE,
        name='encoder',
        dropout=DROPOUT
    )
    
    context = RepeatVector(MAX_MESSAGE_LEN)(encoder_rnn(embedded_input))
    
    # DECODER
    
    last_word_input = Input(
        shape=(MAX_MESSAGE_LEN, ),
        dtype='int32',
        name='last_word_input',
    )
    
    embedded_last_word = shared_embedding(last_word_input)
    # Combines the context produced by the encoder and the last word uttered as inputs
    # to the decoder.
    decoder_input = concatenate([embedded_last_word, context], axis=2)
    
    # return_sequences causes LSTM to produce one output per timestep instead of one at the
    # end of the intput, which is important for sequence producing models.
    decoder_rnn = LSTM(
        CONTEXT_SIZE,
        name='decoder',
        return_sequences=True,
        dropout=DROPOUT
    )
    
    decoder_output = decoder_rnn(decoder_input)
    
    # TimeDistributed allows the dense layer to be applied to each decoder output per timestep
    next_word_dense = TimeDistributed(
        Dense(int(MAX_VOCAB_SIZE / 2), activation='relu'),
        name='next_word_dense',
    )(decoder_output)
    
    next_word = TimeDistributed(
        Dense(MAX_VOCAB_SIZE, activation='softmax'),
        name='next_word_softmax'
    )(next_word_dense)
    
    return Model(inputs=[encoder_input, last_word_input], outputs=[next_word])

s2s_model = create_model()
optimizer = Adam(lr=LEARNING_RATE, clipvalue=5.0)
s2s_model.compile(optimizer='adam', loss='categorical_crossentropy')

In [36]:
def add_start_token(y_array):
    """ Adds the start token to vectors.  Used for training data. """
    return np.hstack([
        START * np.ones((len(y_array), 1)),
        y_array[:, :-1],
    ])

def binarize_labels(labels):
    """ Helper function that turns integer word indexes into sparse binary matrices for 
        the expected model output.
    """
    return np.array([np_utils.to_categorical(row, num_classes=MAX_VOCAB_SIZE)
                     for row in labels])
def respond_to(model, text):
    """ Helper function that takes a text input and provides a text output. """
    input_y = add_start_token(PAD * np.ones((1, MAX_MESSAGE_LEN)))
    idxs = np.array(to_word_idx(text)).reshape((1, MAX_MESSAGE_LEN))
    for position in range(MAX_MESSAGE_LEN - 1):
        prediction = model.predict([idxs, input_y]).argmax(axis=2)[0]
        input_y[:,position + 1] = prediction[position]
    return from_word_idx(model.predict([idxs, input_y]).argmax(axis=2)[0])
def train_mini_epoch(model, start_idx, end_idx):
    """ Batching seems necessary in Kaggle Jupyter Notebook environments, since
        `model.fit` seems to freeze on larger batches (somewhere 1k-10k).
    """
    b_train_y = binarize_labels(train_y[start_idx:end_idx])
    input_train_y = add_start_token(train_y[start_idx:end_idx])
    
    model.fit(
        [train_x[start_idx:end_idx], input_train_y], 
        b_train_y,
        epochs=1,
        batch_size=BATCH_SIZE,
    )
    
    rand_idx = random.sample(list(range(len(test_x))), SUB_BATCH_SIZE)
    print('Test results:', model.evaluate(
        [test_x[rand_idx], add_start_token(test_y[rand_idx])],
        binarize_labels(test_y[rand_idx])
    ))
    
    input_strings = [
        "@AppleSupport I fix I this I stupid I problem I",
        "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service.",
    ]
    
    for input_string in input_strings:
        output_string = respond_to(model, input_string)
        print(f'> "{input_string}"\n< "{output_string}"')

# Training

In [44]:
training_time_limit = 360 * 60  # seconds (notebooks terminate after 1 hour)
start_time = time.time()
stop_after = start_time + training_time_limit

class TimesUpInterrupt(Exception):
    pass

try:
    for epoch in range(20):
        print(f'Training in epoch {epoch}...')
        for start_idx in range(0, len(train_x), SUB_BATCH_SIZE):
            train_mini_epoch(s2s_model, start_idx, start_idx + SUB_BATCH_SIZE)
            if time.time() > stop_after:
                raise TimesUpInterrupt
except KeyboardInterrupt:
    print("Halting training from keyboard interrupt.")
except TimesUpInterrupt:
    print(f"Halting after {time.time() - start_time} seconds spent training.")

Training in epoch 0...
32/32 [==============================] - 8s 217ms/step - loss: 4.1058
Test results: 4.105801105499268
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ hi sorry for , , , , , , , , , , , , , , , , , , , , , , , , , ,"
> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ hi sorry for , , , , , , , , , , , , , , , , , , , , , , , , , ,"
32/32 [==============================] - 7s 218ms/step - loss: 3.7009
Test results: 3.700941562652588
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ hi there , we can help . please dm us a dm us a dm and and ."
> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ hi there , we can help . please dm us a dm us a dm and and ."
32/32 [==============================] - 7s 204ms/step - loss: 3.4247
Test results: 3.4246912002563477
> "@AppleSupport I fix I this I stupid I problem I

32/32 [==============================] - 7s 202ms/step - loss: 2.7139
Test results: 2.7138938903808594
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ hi there , we are sorry to hear this . please dm us your email address and phone number so we can assist you . ^ __unk__"
> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ hi there , we are sorry to hear this . please dm us your email address and phone number so we can assist you . ^ __unk__"
32/32 [==============================] - 7s 214ms/step - loss: 2.7295
Test results: 2.7295308113098145
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ hi there , __unk__ . we have a great __unk__ . ^ __unk__"
> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ hi there , __unk__ . we have a great __unk__ . ^ __unk__"
32/32 [==============================] - 7s 205ms/step - loss: 2.6411


> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ hi there . please dm us your email address and we'll take a look into this . ^ rr"
32/32 [==============================] - 6s 201ms/step - loss: 2.4603
Test results: 2.4603006839752197
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ hi there , we are sorry to hear this . please dm us your email address so we can look into this for you . ^ hp"
> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ hi there , we are sorry to hear this . please dm us your email address so we can look into this for you . ^ hp"
32/32 [==============================] - 6s 201ms/step - loss: 2.4350
Test results: 2.4349539279937744
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ hi there , we ’ re sorry to hear this . please dm us your email address and we'll reach out right away ."
> "@AmazonHelp I 

> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ hi there , we are here to help . please dm us your email address so we can look into this for you ."
32/32 [==============================] - 7s 211ms/step - loss: 2.3291
Test results: 2.3290913105010986
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ hi there ! can you dm us your account's email address ? we'll take a look under the hood / au __url__"
> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ hi there , we are sorry to hear this . please dm us your tracking number and contact number . ^ mm __url__"
32/32 [==============================] - 7s 213ms/step - loss: 2.3118
Test results: 2.3117711544036865
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ we can help . please dm us with your email address so we can connect ."
> "@AmazonHelp I hadnt expected that such a bi

> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ hi there , can you dm us your gamertag and some details about what happened ? ^ ez"
32/32 [==============================] - 7s 218ms/step - loss: 2.2720
Test results: 2.2720255851745605
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ we want to help . which iphone are you running ?"
> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ hi there , please dm us your tracking number and contact info . thanks !"
32/32 [==============================] - 6s 201ms/step - loss: 2.2234
Test results: 2.223447322845459
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ we want to help . which device are you using ?"
> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ hi there , we are sorry to hear this . can you please dm

...

32/32 [==============================] - 6s 201ms/step - loss: 1.9538
Test results: 1.9538331031799316
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ we want to help . please dm us the device model you have and the ios version installed . __url__"
> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ i'm sorry for the delay . please reach out to us here : __url__ and we'll be happy to help . ^ sg"
32/32 [==============================] - 6s 201ms/step - loss: 1.8340
Test results: 1.8340083360671997
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ we want to help . let's take a look at this together . which ios version are you using ?"
> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ hi there , we are sorry to hear this . please dm us your tracking number , delivery address and phone number so we can look into this for"
32/3

32/32 [==============================] - 6s 202ms/step - loss: 1.8917
Test results: 1.8917073011398315
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ we want to help . please dm us the ios version you have and we'll continue there . __url__"
> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ i'm sorry to hear this . please reach out to us here : __url__ and we'll look into this . ^ rs"
32/32 [==============================] - 7s 203ms/step - loss: 1.9378
Test results: 1.937806487083435
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ we can help . dm us and we'll go from there . __url__"
> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ i'm sorry to hear this ! we'd like to help . please reach out to us here : __url__ ^ jm"
32/32 [==============================] - 7s 207ms/step - loss: 1.9042
Test results: 1.9042254686355

> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ i'm sorry for the trouble with the delivery . please report this to our support team here : __url__ and we'll look into it . ^ hn"
32/32 [==============================] - 7s 203ms/step - loss: 1.9264
Test results: 1.9264328479766846
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ we want to help . dm us and let us know which country you're in . __url__"
> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ i'm sorry for the trouble ! we'd like to help . please contact us here : __url__ so we can look into this . ^ rs"
32/32 [==============================] - 7s 204ms/step - loss: 1.9006
Test results: 1.9005588293075562
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ we want to help . which iphone and ios version are you using ?"
> "@AmazonHelp I hadnt expected that such a bi

32/32 [==============================] - 6s 202ms/step - loss: 1.9086
Test results: 1.9085568189620972
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ we want to help . dm us and we'll go from there . __url__"
> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ i'm sorry for the trouble with your order ! we'd like to help . please reach out to us here : __url__ ^ af"
32/32 [==============================] - 7s 204ms/step - loss: 1.9202
Test results: 1.9202122688293457
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ we want to help . send us a dm with your current ios version and we'll go from there . __url__"
> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ i'm sorry for the trouble with your order ! we'd like to help . please reach out to us here : __url__ ^ af"
32/32 [==============================] - 7s 221ms/step - lo

32/32 [==============================] - 7s 203ms/step - loss: 1.9082
Test results: 1.9081976413726807
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ we want to help . send us a dm with the country that you are located in to continue . __url__"
> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ i'm sorry for the trouble ! we'd like to help . please reach out to us here : __url__ ^ ha"
32/32 [==============================] - 7s 202ms/step - loss: 1.9591
Test results: 1.9590848684310913
> "@AppleSupport I fix I this I stupid I problem I"
< "@__sn__ we'd like to help . please dm us which iphone and ios version you're using . __url__"
> "@AmazonHelp I hadnt expected that such a big brand like amazon would have such a poor customer service."
< "@__sn__ i'm sorry for the trouble with your order ! we'd like to help . please reach out to us here : __url__ so we can look into this ."
32/32 [====================

In [46]:
# s2s_model.save("./s2s_model.h5")

In [49]:
model = keras.models.load_model(
    './s2s_model.h5')
respond_to(model, "@AppleSupport testing testing!")

"@__sn__ we want to help . dm us and we'll go from there . __url__"

In [48]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 last_word_input (InputLayer)   [(None, 30)]         0           []                               
                                                                                                  
 encoder_input (InputLayer)     [(None, 30)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 30, 100)      819200      ['encoder_input[0][0]',          
                                                                  'last_word_input[0][0]']        
                                                                                                  
 encoder (LSTM)                 (None, 100)          80400       ['embedding[0][0]']        